In [1]:
pip install pandas

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: C:\Users\Shei\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
import os
import pandas as pd

In [3]:
def load_day_tabular_data(day_path, verbose=True):
    dfs = []

    # Map folder to date
    date_map = {
        '7_24_data': '2019-07-24',
        '10_19_data': '2019-10-19',
        '11_10_data': '2019-11-10'
    }
    folder_name = os.path.basename(day_path)
    default_date = date_map.get(folder_name, None)

    for file in os.listdir(day_path):
        if file.endswith('.csv') and file.startswith('location'):
            location = file.replace('.csv', '')
            file_path = os.path.join(day_path, file)

            df = pd.read_csv(file_path)

            if 'time' not in df.columns:
                raise ValueError(f"'time' column missing in {file_path}")

            # Special handling for 7_24_data: time is only HH:MM:SS
            if folder_name == '7_24_data':
                if default_date is None:
                    raise ValueError("Date for 7_24_data not found in date_map.")
                # Combine date and time string
                df['parsed_time'] = pd.to_datetime(default_date + ' ' + df['time'].astype(str), format='%Y-%m-%d %H:%M:%S', errors='coerce')
            else:
                df['parsed_time'] = pd.to_datetime(df['time'], errors='coerce')

            # Identify invalid rows
            invalid_rows = df[df['parsed_time'].isna()]
            num_invalid = len(invalid_rows)

            if num_invalid > 0:
                if verbose:
                    print(f"\nFound {num_invalid} invalid 'time' entries in {file_path}")
                    print("Sample invalid values:")
                    print(invalid_rows['time'].dropna().unique()[:5])

                invalid_dir = '../dataset/b_merged_tabular_data'
                invalid_save_path = os.path.join(
                                        invalid_dir, f'invalid_{os.path.basename(day_path)}_{location}.csv'
                                    )               
                invalid_rows.to_csv(invalid_save_path, index=False)
                print(f"Saved invalid entries to: {invalid_save_path}")

            df = df.dropna(subset=['parsed_time'])
            df['time'] = df['parsed_time']
            df.drop(columns=['parsed_time'], inplace=True)
            df['location'] = location

            dfs.append(df)

    if not dfs:
        raise ValueError(f"No location CSVs found in {day_path}")

    merged_df = pd.concat(dfs, ignore_index=True)
    return merged_df

In [4]:
dataset_root = '../dataset/a_raw'

# Load per-day merged tabular data
day1_7_24_df = load_day_tabular_data(os.path.join(dataset_root, '7_24_data'))
day2_10_19_df = load_day_tabular_data(os.path.join(dataset_root, '10_19_data'))
day3_11_10_df = load_day_tabular_data(os.path.join(dataset_root, '11_10_data'))



Found 1 invalid 'time' entries in ../dataset/a_raw\10_19_data\location5.csv
Sample invalid values:
['20']
Saved invalid entries to: ../dataset/b_merged_tabular_data\invalid_10_19_data_location5.csv


In [5]:
# Display samples of the merged DataFrames

print("Day 1 sample:")
display(day1_7_24_df.head())
display(day1_7_24_df.tail())

print("\nDay 2 sample:")
display(day2_10_19_df.head())
display(day2_10_19_df.tail())

print("\nDay 3 sample:")
display(day3_11_10_df.head())
display(day3_11_10_df.head())


Day 1 sample:


,pm2.5,pm10,time,location
0,18.1,23.5,2019-07-24 13:32:18,location1
1,18.2,24.0,2019-07-24 13:32:19,location1
2,18.3,24.3,2019-07-24 13:32:20,location1
3,18.3,24.2,2019-07-24 13:32:21,location1
4,18.4,24.5,2019-07-24 13:32:22,location1


,pm2.5,pm10,time,location
64705,17.0,34.2,2019-07-24 17:49:26,location8
64706,17.3,35.4,2019-07-24 17:49:27,location8
64707,17.4,36.1,2019-07-24 17:49:28,location8
64708,17.1,34.9,2019-07-24 17:49:29,location8
64709,17.2,35.4,2019-07-24 17:49:30,location8



Day 2 sample:


,pm2.5,pm10,time,temperature,humidity,location
0,52.6,68.6,2019-10-19 10:06:23.748128,21.939803,55.631342,location1
1,52.7,68.2,2019-10-19 10:06:24.665349,21.969177,55.622187,location1
2,53.2,68.1,2019-10-19 10:06:25.582539,21.955825,55.588617,location1
3,53.3,68.7,2019-10-19 10:06:26.499647,21.955825,55.599298,location1
4,53.4,69.3,2019-10-19 10:06:27.722145,21.955825,55.634394,location1


,pm2.5,pm10,time,temperature,humidity,location
123815,65.5,104.9,2019-10-19 14:01:52.015898,22.308309,49.213397,location8
123816,65.3,102.5,2019-10-19 14:01:53.273897,22.278935,49.245441,location8
123817,64.8,105.1,2019-10-19 14:01:54.211029,22.321660,49.323262,location8
123818,65.4,105.7,2019-10-19 14:01:55.143086,22.335012,49.373617,location8
123819,65.5,105.9,2019-10-19 14:01:56.094252,22.308309,49.394980,location8



Day 3 sample:


,pm2.5,pm10,time,temperature,humidity,location
0,43.9,75.9,2019-11-10 09:56:37.864574,20.423056,51.949340,location1
1,44.4,76.5,2019-11-10 09:56:38.815773,20.396353,51.964599,location1
2,44.6,76.5,2019-11-10 09:56:39.743464,20.423056,52.004273,location1
3,44.4,75.4,2019-11-10 09:56:40.681302,20.383001,52.040894,location1
4,44.6,76.3,2019-11-10 09:56:41.909214,20.409705,52.141604,location1


,pm2.5,pm10,time,temperature,humidity,location
0,43.9,75.9,2019-11-10 09:56:37.864574,20.423056,51.949340,location1
1,44.4,76.5,2019-11-10 09:56:38.815773,20.396353,51.964599,location1
2,44.6,76.5,2019-11-10 09:56:39.743464,20.423056,52.004273,location1
3,44.4,75.4,2019-11-10 09:56:40.681302,20.383001,52.040894,location1
4,44.6,76.3,2019-11-10 09:56:41.909214,20.409705,52.141604,location1


In [6]:
# Save the merged DataFrames to CSV files
output_dir = '../dataset/b_merged_tabular_data'

day1_7_24_df.to_csv(os.path.join(output_dir, 'merged_7_24.csv'), index=False)
day2_10_19_df.to_csv(os.path.join(output_dir, 'merged_10_19.csv'), index=False)
day3_11_10_df.to_csv(os.path.join(output_dir, 'merged_11_10.csv'), index=False)

print("Saved all merged day CSVs")

Saved all merged day CSVs
